## Implementing gradient descent for multiple regression

import libraries and import dataset

In [2]:
import graphlab as gl
import numpy as np
sales = gl.SFrame('kc_house_data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\WINDOWS\TEMP\graphlab_server_1474611125.log.0


This non-commercial license of GraphLab Create for academic use is assigned to luwang@asu.edu and will expire on May 26, 2017.


## convert to Numpy

Now we will write a function that will accept an SFrame, a list of feature names (e.g. ['sqft_living', 'bedrooms']) and an target feature e.g. ('price') and will return two things:

* A numpy matrix whose columns are the desired features plus a constant column (this is how we create an 'intercept')

* A numpy array containing the values of the output

In [15]:
def get_np_data(data,features,output):
    data['constant']=1
    features=['constant']+features
    features_sframe=data[features]
    feature_matrix=features_sframe.to_numpy()
    output_sarray=data[output]
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)
    

In [16]:
(example_features, example_output) = get_np_data(sales, ['sqft_living'], 'price') # the [] around 'sqft_living' makes it a list
print example_features[0,:] # this accesses the first row of the data the ':' indicates 'all columns'
print example_output[0] # and the corresponding output

[  1.00000000e+00   1.18000000e+03]
221900.0


# Predicting output given regression weights

In [17]:
my_weights = np.array([1., 1.]) # the example weights
my_features = example_features[0,] # we'll use the first data point
predicted_value = np.dot(my_features, my_weights)
print predicted_value

1181.0


predict_output function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [20]:
def predict_output(feature_matrix, weights):
    predictions=np.dot(feature_matrix,weights)
    return(predictions)

test function

In [21]:
test_predictions = predict_output(example_features, my_weights)
print test_predictions[0] # should be 1181.0
print test_predictions[1] # should be 2571.0

1181.0
2571.0


# Computing the Derivative

We are now going to move to computing the derivative of the regression cost function. Recall that the cost function is the sum over the data points of the squared difference between an observed output and a predicted output.

Since the derivative of a sum is the sum of the derivatives we can compute the derivative for a single data point and then sum over data points. We can write the squared difference between the observed output and predicted output for a single point as follows:

(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)^2

Where we have k features and a constant. So the derivative with respect to weight w[i] by the chain rule is:

2\*(w[0]\*[CONSTANT] + w[1]\*[feature_1] + ... + w[i] \*[feature_i] + ... +  w[k]\*[feature_k] - output)\* [feature_i]

The term inside the paranethesis is just the error (difference between prediction and output). So we can re-write this as:

2\*error\*[feature_i]

That is, the derivative for the weight for feature i is the sum (over data points) of 2 times the product of the error and the feature itself. In the case of the constant then this is just twice the sum of the errors!

Recall that twice the sum of the product of two vectors is just twice the dot product of the two vectors. Therefore the derivative for the weight for feature_i is just two times the dot product between the values of feature_i and the current errors. 

With this in mind complete the following derivative function which computes the derivative of the weight given the value of the feature (over all data points) and the errors (over all data points).

In [23]:
def feature_derivative(errors, feature):
    # Assume that errors and feature are both numpy arrays of the same length (number of data points)
    # compute twice the dot product of these vectors as 'derivative' and return the value
    derivative=np.dot(2*errors,feature)
    return(derivative)

Test the function

In [26]:
(example_features, example_output) = get_np_data(sales, ['sqft_living'], 'price') 
my_weights = np.array([0., 0.]) # this makes all the predictions 0
test_predictions = predict_output(example_features, my_weights) 
# just like SFrames 2 numpy arrays can be elementwise subtracted with '-': 
errors = test_predictions - example_output # prediction errors in this case is just the -example_output
feature = example_features[:,0] # let's compute the derivative with respect to 'constant', the ":" indicates "all rows"
derivative = feature_derivative(errors, feature)
print derivative
print -np.sum(example_output)*2 # should be the same as derivative

-23345850022.0
-23345850022.0


# Gradient Descent

Now we will write a function that performs a gradient descent. The basic premise is simple. Given a starting point we update the current weights by moving in the negative gradient direction. Recall that the gradient is the direction of *increase* and therefore the negative gradient is the direction of *decrease* and we're trying to *minimize* a cost function. 

The amount by which we move in the negative gradient *direction*  is called the 'step size'. We stop when we are 'sufficiently close' to the optimum. We define this by requiring that the magnitude (length) of the gradient vector to be smaller than a fixed 'tolerance'.

With this in mind, complete the following gradient descent function below using your derivative function above. For each step in the gradient descent we update the weight for each feature befofe computing our stopping criteria

In [28]:
from math import sqrt # recall that the magnitude/length of a vector [g[0], g[1], g[2]] is sqrt(g[0]^2 + g[1]^2 + g[2]^2)

In [51]:
def regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        # compute the predictions based on feature_matrix and weights using your predict_output() function
        predictions=predict_output(feature_matrix, weights)
        # compute the errors as predictions - output
        error=predictions-output
        gradient_sum_squares = 0 # initialize the gradient sum of squares
        # while we haven't reached the tolerance yet, update each feature's weight
        for i in range(len(weights)): # loop over each weight
            # Recall that feature_matrix[:, i] is the feature column associated with weights[i]
            # compute the derivative for weight[i]:
            derivative=feature_derivative(error, feature_matrix[:,i])
            # add the squared value of the derivative to the gradient sum of squares (for assessing convergence)
            gradient_sum_squares=gradient_sum_squares+derivative*derivative
            # subtract the step size times the derivative from the current weight
            weights[i]=weights[i]-step_size*derivative
        # compute the square-root of the gradient sum of squares to get the gradient matnigude:
        gradient_magnitude = sqrt(gradient_sum_squares)
        if gradient_magnitude < tolerance:
            converged = True
    return(weights)

# Running the Gradient Descent as Simple Regression

In [116]:
train_data,test_data = sales.random_split(.8,seed=0)

In [117]:
# let's test out the gradient descent
simple_features = ['sqft_living']
my_output = 'price'
(simple_feature_matrix, output) = get_np_data(train_data, simple_features, my_output)
initial_weights = np.array([-47000., 1.])
step_size = 7e-12
tolerance = 2.5e7

In [118]:
weights=regression_gradient_descent(simple_feature_matrix, output, initial_weights, step_size, tolerance)

In [119]:
(test_simple_feature_matrix, test1_output) = get_np_data(test_data, simple_features, my_output)

In [120]:
predictions1=predict_output(test_simple_feature_matrix, weights)

In [121]:
weights

array([-46999.88716555,    281.91211912])

# Running a multiple regression

In [93]:
model_features = ['sqft_living', 'sqft_living15'] # sqft_living15 is the average squarefeet for the nearest 15 neighbors. 
my_output = 'price'
(feature_matrix, output) = get_np_data(train_data, model_features, my_output)
initial_weights = np.array([-100000., 1., 1.])
step_size = 4e-12
tolerance = 1e9

In [94]:
weights=regression_gradient_descent(feature_matrix, output, initial_weights, step_size, tolerance)

In [96]:
(test_feature_matrix, test2_output) = get_np_data(test_data, model_features, my_output)

In [97]:
predictions2=predict_output(test_feature_matrix, weights)

In [98]:
predictions2

array([ 366651.41203656,  762662.39786164,  386312.09499712, ...,
        682087.39928241,  585579.27865729,  216559.20396617])

In [113]:
test_data[0]

{'bathrooms': 1.0,
 'bedrooms': 3.0,
 'condition': 4L,
 'constant': 1L,
 'date': datetime.datetime(2014, 5, 28, 0, 0, tzinfo=GMT +0.0),
 'floors': '1.5',
 'grade': 7L,
 'id': '0114101516',
 'lat': 47.75584254,
 'long': -122.22874498,
 'price': 310000.0,
 'sqft_above': 1430L,
 'sqft_basement': 0L,
 'sqft_living': 1430.0,
 'sqft_living15': 1780.0,
 'sqft_lot': 19901L,
 'sqft_lot15': 12697.0,
 'view': 0L,
 'waterfront': 0L,
 'yr_built': 1927L,
 'yr_renovated': 0L,
 'zipcode': '98028'}

## compute the rss for both models

In [109]:
rss1=((predictions1-test_data['price'])*(predictions1-test_data['price'])).sum()

In [110]:
rss2=((predictions2-test_data['price'])*(predictions2-test_data['price'])).sum()

In [111]:
rss1

275400047593155.94

In [112]:
rss2

270263446465244.06